In [ ]:
# Importation de modules

# Normaliser les chaînes de caractère
from time import sleep
import unicodedata as uni
# Écrire des Regex
import re
# Modifier des fichiers sur le système d'exploitation
import os
# Lire et écrire du .csv
import csv as csvlib
# Comparer des chaînes de caractères
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# Lire du .xml
import xml.etree.ElementTree as ET
# Définition du namespace du xml
ns = "http://www.tei-c.org/ns/1.0"
from itertools import combinations
from tqdm.auto import tqdm
from math import factorial
from numpy import array, mean, percentile
import pandas as pd



In [ ]:
# Fonction pour supprimer les diacritiques

def strip_accents(string):
   return ''.join(c for c in uni.normalize('NFD', string)
                  if uni.category(c) != 'Mn')

In [ ]:
# Extraire les phrases et créer les squelettes textuels

# initier un compteur de phrases
id_phrase = 0

# créer un dictionnaire nodes[id_text + "ph" + id_phrase] = [id_text, id_phrase, text, label]
nodes = {}
# créer un dictionnaire edges[source] = [target, edge_type, lev_distance]
edges = {}

min_phr_len = 20
max_phr_len = 2000

for root_path, _, files in os.walk("mazarinades"):
    for name in files:
        if(not name.endswith(".xml")):
            continue
        # pour chaque fichier .xml du dossier mazarinades
        with open(os.path.join(root_path, name), 'r', encoding='utf-8') as f:
            xml = f.read()
            root = ET.fromstring(xml)
            id_phrase = 0

            # récupérer le titre du document pour l'afficher en label des noeuds
            title = root.find(".//{" + ns + "}title").text
            try:
                label = re.sub(r'\s+', ' ', title).replace('\n', '')
            except:
                label = "no_title"

            # récupérer l'identifiant dans id_text
            id_text = name.replace(".xml", "")
            # insérer le noeud titre dans le dictionnaire de noeuds
            nodes[id_text + "ph" +
                  str(id_phrase)] = [id_text, str(id_phrase), "null", label]

            # extraire le texte et supprimer les balises, les espaces multiples, les tirets, les diacritiques et mettre tout en minuscules
            full_text = re.sub(
                r'\s+', ' ', (re.sub(r'<.+?>|<\/?|\/?>', ' ', xml.split("body>")[1])))
            full_text = strip_accents(full_text.replace("¬", "").lower())

            # créer un noeud pour chaque phrase qui dépasse 20 caractères
            phrases = re.findall("[^\.\?\!]{" + str(min_phr_len) + ",}", full_text)
            for phrase in phrases:
                if id_text == "Moreau1906_GBOOKS":
                    print(phrase)
                id_phrase += 1
                nodes[id_text + "ph" + str(id_phrase)] = [id_text,
                                                     id_phrase, phrase, "null"]
                # créer un lien avec la phrase précédente
                edges[id_text + "ph" +
                      str(id_phrase - 1)] = [id_text + "ph" + str(id_phrase), "precedence", "null"]

# vérifier le nombre de phrases
print(len(nodes))
                    

In [ ]:
# Créer les liens de proximité entre phrases

# créer une liste de textes déjà évalués
analysed_texts = []
min_lev_distance = 0.80
graduations = []

# définir des graduations en fonction de la taille minimale, maximale et de l'écart de levensthein minimal
i = min_phr_len
while i < max_phr_len:
    graduations.append(round(i))
    i = i/min_lev_distance
graduations.append(max_phr_len)

# répartir les noeuds dans ces groupes
groups = [[(id, node) for id, node in nodes.items() if graduations[i] <
           len(node[2]) < graduations[i+2]] for i in range(len(graduations)-2)]

tot = sum([factorial(len(group)) // (2*factorial(len(group) - 2)) for group in groups])
with tqdm(total = tot) as pbar:
    for group in groups:
        for (id1, node1), (id2, node2) in combinations(group, 2):
            pbar.update(1)

'''
group_num = 10
# définir la moyenne de la longueur des phrases
avg = mean([len(node[2]) for _, node in nodes.items()])
# récupérer la longueur de chaque phrase
data = [len(node[2]) for _, node in nodes.items()]
deciles = percentile(data, range(0, 101, int((1/group_num)*100)), interpolation='midpoint')
groups = [[(id, node) for id, node in nodes.items() if deciles[i] < len(node[2]) < deciles[i+2]] for i in range(group_num-1)]
print(len(groups[0]))

with tqdm(total=factorial(len(nodes)) // (2*factorial(len(nodes) - 2))) as pbar:
    for (id1, node1), (id2, node2) in combinations(nodes.items(), 2):
        pbar.update(1)
'''

'''
# pour chaque phrase du corpus
for id, node in nodes.items():
    # enregistrer le texte comme analysé
    analysed_texts.append(node[0])
    # si la phrase analysée n'est pas un titre
    if (node[2] != "null"):
        # calculer une distance de levensthein pour toutes les phrases pas encore analysées
        for cur_id, cur_node in nodes.items():
            if (cur_node[0] not in analysed_texts and cur_node[2] != "null"):
                lev_distance = fuzz.ratio(node[2], cur_node[2]) / 100
        # créer des liens avec les phrases qui sont proches à 80%
                if (lev_distance >= 0.70):
                    print(node[2] + " : " + cur_node[2] +
                          " : " + str(lev_distance))
                    edges[id] = [cur_id, "proximity", lev_distance]
'''


In [ ]:
# Verser le contenu dans un fichier maz_phrases.csv


